# OpenStreetMap Data Case Study

## Map parser
Use mapparse.py to fnamed unique tags

In [2]:
import xml.etree.cElementTree as ET
from collections import defaultdict
import csv, sqlite3
import re
import pprint

OSMFILE = "tampa_florida.osm"

In [3]:
#%%writefile mapparser.py

def count_tags(filename):
    tag_count = {}
    for _, element in ET.iterparse(filename, events=("start",)):
        add_tag(element.tag, tag_count)
    return tag_count

def add_tag(tag, tag_count):
    if tag in tag_count:
        tag_count[tag] += 1
    else:
        tag_count[tag] = 1

In [4]:
tags = count_tags(OSMFILE)
pprint.pprint(tags)

{'bounds': 1,
 'member': 31857,
 'nd': 1957582,
 'node': 1655566,
 'osm': 1,
 'relation': 1252,
 'tag': 1131585,
 'way': 182866}


## Tags
Use tags.py to fnamed patterns in the tags.

In [5]:
#%%writefile tags.py

lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

def key_type(element, keys):
    if element.tag == "tag":
        key = element.get("k")
        #print key
        if problemchars.search(key):
            keys['problemchars'] += 1
            #print '--> problemchars'
        elif lower_colon.search(key):
            keys['lower_colon'] += 1
            #print '--> lower_colon'
        elif lower.search(key):
            keys['lower'] += 1
            #print '--> lower'
        else:
            keys['other'] += 1
            #print '--> other'
    return keys

def process_map(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)

    return keys


In [6]:
keys = process_map(OSMFILE)
pprint.pprint(keys)

{'lower': 575997, 'lower_colon': 520908, 'other': 34675, 'problemchars': 5}


## Users
fnamed user ids by using users.py

In [7]:
#%%writefile users.py

def get_user(element):
    return

def process_map(filename):
    users = set()
    for _, element in ET.iterparse(filename):
        if 'uid' in element.attrib:
            users.add(element.get('uid'))

    return users

In [8]:
users = process_map(OSMFILE)
pprint.pprint(users)

set(['1',
     '100042',
     '1007528',
     '1009527',
     '1012362',
     '101999',
     '102411',
     '102582',
     '102999',
     '103095',
     '103107',
     '103253',
     '10344',
     '103464',
     '103574',
     '1037425',
     '104473',
     '104519',
     '104962',
     '1051550',
     '1058308',
     '105839',
     '1058666',
     '10786',
     '108295',
     '1083136',
     '1086615',
     '1087647',
     '1089595',
     '109035',
     '10927',
     '109925',
     '110046',
     '110126',
     '110263',
     '1103752',
     '110639',
     '1108251',
     '1110270',
     '1110420',
     '11126',
     '1118605',
     '113450',
     '1139460',
     '114161',
     '114388',
     '1149057',
     '11515',
     '115918',
     '116044',
     '1161073',
     '1164',
     '118021',
     '1180961',
     '118134',
     '1185729',
     '118613',
     '118856',
     '1198074',
     '1199709',
     '120468',
     '1207207',
     '1208453',
     '121241',
     '1214881',
     '12198

## Audit
Audit and clean data with audit.py

In [9]:
#%%writefile audit.py
import re

street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)
direction_re = re.compile(r'\w*(North|South|East|West|Northeast|Northwest|Southeast|Southwest|S|NE|W|N|E|SE|N.)$')

direction_mapping = {"N":"North",
                    "S":"South",
                    "NE":"Northeast",
                    "W":"West",
                    "E":"East",
                    "SE": "Southeast"}

expected = ["Passage","Cutoff","Bridge","Crossing","Lane","Way","Run","Loop","Plaza","Causeway","Terrace","Highway","Bayway","Circle","Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road",
            "Trail", "Parkway", "Commons"]

city_mapping = {'St. Petersburg, FL':'St. Petersburg',
                'St Petersburg ': 'St. Petersburg',
                'St Pete Beach': 'St. Pete Beach',
                'SPRING HILL': 'Spring Hill',
                'sarasota': 'Sarasota',
                'St Petersburg': 'St. Petersburg',
                'lutz': 'Lutz',
                'spring hill': 'Spring Hill',
                'Zephyhills': 'Zephyrhills',
                'port richey': 'Port Richey',
                'Miakka': 'Old Myakka',
                'Saint Petersburg': 'St. Petersburg',
                'Seminole ': 'Seminole',
                'Land O Lakes': "Land O' Lakes",
                'Tampa ': 'Tampa',
                'St Petersbug': 'St. Petersburg',
                'hudson': 'Hudson',
                'Land O Lakes, FL': "Land O' Lakes",
                'Clearwarer Beach': 'Clearwater Beach',
                'Palm Harbor, Fl.': 'Palm Harbor',
                'tampa': 'Tampa'}

cities = ['Indian Shores',"Land O' Lakes", 'Pinellas Park',
          'Largo', 'Trinity', 'St. Petersburg', 'Bay Pines',
          'Treasure Island', 'Indian Rocks Beach', 'Apollo Beach',
          'Palm Harbor', 'Temple Terrace','Tampa', 'St. Pete Beach',
          'Lakeland', 'Old Myakka', 'Plant City','Dunedin', 'South Highpoint', 
          'Madeira Beach', 'Gulfport', 'Lakewood Ranch', 'Longboat Key', 'Brandon',
          'Clearwater Beach', 'Verna', 'Seminole', 'Dade City', 'Feather Sound', 'Redington Shores',
          'Gandy', 'South Pasadena', 'Cortez Village', 'Safety Harbor', 'San Antonio',
          'Anna Maria', 'Bradenton Beach', 'Palm Harbor',
          'Wesley Chapel', 'Tarpon Springs', 'New Port Richey',
          'St. Petersburg', 'Port Richey', 'Clearwater', 'Pasadena', 'Redington Beach',
          'Holiday', 'Sarasota', 'Lutz', 'Wimauma', 'Parrish', 'Zephyrhills', 'Shady Hills',
          'Thonotosassa', 'Belleair', 'Belleair Beach', 'Ellenton', 'Ruskin', 'Oldsmar',
          'Valrico', 'Kenneth City', 'Hudson', 'Riverview',
          'Bradenton', 'Odessa', 'Gibsonton', 'Lithia',
          'Palmetto', 'Pass-a-Grille Beach', 'Spring Hill',
          'Holmes Beach', 'Dover', 'Seffner', 'Sun City Center', 'Saint Leo']

mapping = { "St": "Street",
            "St.": "Street",
            "Ave": "Avenue",
            "Rd": "Road",
            "Rd.": "Road",
            "Dr": "Drive",
            "Av": "Avenue",
            "AVE": "Avenue",
            "Blvd": "Boulevard",
            "Cir": "Circle",
            "Hwy": "Highway",
            "Blvd.": "Boulevard",
            "Pkwy": "Parkway",
            "dr": "Drive",
            "Dr.": "Drive",
            "Ave.": "Avenue",
            "Pl": "Place",
            "Cswy": "Causeway",
            "Plz": "Plaza",
            "Ct": "Court",
            "Pky": "Parkway",
            "Ln": "Lane",
            "st": "Street",
            "road": "Road",
            "drive": "Drive",
            "lane": "Lane"
            }

def clean_state(state_name):
    if state_name != 'FL':
        state_name = 'FL'
    return state_name

def audit_state(state_types, state_name):
    if state_name != 'FL':
        state_types[state_name] += 1

def audit_street_type(street_types, street_name):
    try:
        comma_index = street_name.index(',')
        street_name = remove(street_name,comma_index)
    except:
        pass
        #print street_name
    try:
        pound_index = street_name.index('#')
        street_name = remove(street_name, pound_index)
    except:
        pass
    
    try:
        suite_index = street_name.index('Suite')
        street_name = remove(street_name, suite_index)
    except:
        pass

    end_direction = direction_re.search(street_name)
    if end_direction:
        street_name = street_name[:-len(end_direction.group(0))]
        street_name = end_direction.group(0) + " " + street_name
        try:
            street_name = update_direction(street_name,direction_mapping)
        except:
            pass
    
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)
    return street_name

def audit_city(invalid_cities, city_name):
    if city_name not in cities and city_name not in city_mapping:
        invalid_cities[city_name] += 1   

def clean_city(city_name):
    if city_name not in cities and city_name in city_mapping:
        city_name = city_mapping[city_name]
    return city_name

def audit_zipcode(invalid_zipcodes, zipcode):
    if not re.match(r'^\d{5}$', zipcode):
        invalid_zipcodes[zipcode] += 1
    return zipcode
             
def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")

def is_state(elem):
    return (elem.attrib['k'] == "addr:state")
    
def is_city(elem):
    return (elem.attrib['k'] == "addr:city")

def is_zipcode(elem):
    return 'zip' in elem.attrib['k']

def audit(osmfile):
    osm_file = open(osmfile, "r")
    
    street_types = defaultdict(set)
    city_types = defaultdict(int)
    zipcode_types = defaultdict(int)
    
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
                elif is_zipcode(tag):
                    audit_zipcode(zipcode_types, tag.attrib['v'])
                elif is_city(tag):
                    audit_city(city_types, tag.attrib['v'])

    return street_types, zipcode_types


def update_name(name, mapping):
    name_array = name.split(' ')
    last = name_array[-1]
    name_array[-1] = mapping[last]
    joined = ' '.join(name_array)
    return joined

def update_direction(name, mapping):
    name_array = name.split(' ')
    first = name_array[0]
    name_array[0] = mapping[first]
    return ' '.join(name_array)

def remove(name,index):
    subname = name[:index]
    return subname

In [10]:
st_types, invalid_zips = audit(OSMFILE)

for st_type, ways in st_types.iteritems():
    for name in ways:
        try:
            better_name = update_name(name, mapping)
            #print name, "=>", better_name
        except:
            print name
            
print invalid_zips

State Road 52
SR 52
FL 52
Boulevard of the Arts
SR 56
FL 56
N U.S. Hwy 41
US 41
N US Highway 41
N US 41
Main St m104
6010 US-301
US-301
State Road 64
U.S.19
US 98 Bypass
Corey Ave  St Pete Beach
4th Street Notth
US 92
12000 US Highway 92
North Avenue Republica de Cuba
Avenue Republica de Cuba
West Brandon Blvd (S.R. 60)
S Howard Av 105
North Westshore Bolevard
8492 Manatee Bay Dr Tampa
Avenue B
SR 580
State Road 580
Avenue F
US 301
US Highway 301
S US Highway 301
South US Highway 301
us Highway 301
CR 672
University Square Mall
E FL 70
Lakewood Main St Ste 102
FL 60
US-19
W Swann Av Prkg
US 98
US Highway 19
US 19
us 19
3001 US Hwy 19
U.S. 19
US Hwy 19
28519 State Road 54
FL 54
SR 54
State Road 54
Sunshine skyway
Bay Esplanade
US 301 (FL)
US 19 (FL)
S Fort Harrison
Broadway
defaultdict(<type 'int'>, {'33548:33556': 1, '34234:34243': 4, '33701:33704': 4, '33760;33764': 1, '33540; 33849': 2, '34683:34689': 2, '33764; 33756': 8, '33765; 33755': 18, '33569;33547': 2, '33765; 33759; 33765': 

In [ ]:
#%%writefile data.py
#!/usr/bin/env python

"""
After auditing is complete the next step is to prepare the data to be inserted into a SQL database.
To do so you will parse the elements in the OSM XML file, transforming them from document format to
tabular format, thus making it possible to write to .csv files.  These csv files can then easily be
imported to a SQL database as tables.

The process for this transformation is as follows:
- Use iterparse to iteratively step through each top level element in the XML
- Shape each element into several data structures using a custom function
- Utilize a schema and validation library to ensure the transformed data is in the correct format
- Write each data structure to the appropriate .csv files

We've already provided the code needed to load the data, perform iterative parsing and write the
output to csv files. Your task is to complete the shape_element function that will transform each
element into the correct format. To make this process easier we've already defnameed a schema (see
the schema.py file in the last code tab) for the .csv files and the eventual tables. Using the 
cerberus library we can validate the output against this schema to ensure it is correct.

## Shape Element Function
The function should take as input an iterparse Element object and return a dictionary.

### If the element top level tag is "node":
The dictionary returned should have the format {"node": .., "node_tags": ...}

The "node" field should hold a dictionary of the following top level node attributes:
- id
- user
- uid
- version
- lat
- lon
- timestamp
- changeset
All other attributes can be ignored

The "node_tags" field should hold a list of dictionaries, one per secondary tag. Secondary tags are
child tags of node which have the tag name/type: "tag". Each dictionary should have the following
fields from the secondary tag attributes:
- id: the top level node id attribute value
- key: the full tag "k" attribute value if no colon is present or the characters after the colon if one is.
- value: the tag "v" attribute value
- type: either the characters before the colon in the tag "k" value or "regular" if a colon
        is not present.

Additionally,

- if the tag "k" value contains problematic characters, the tag should be ignored
- if the tag "k" value contains a ":" the characters before the ":" should be set as the tag type
  and characters after the ":" should be set as the tag key
- if there are additional ":" in the "k" value they and they should be ignored and kept as part of
  the tag key. For example:

  <tag k="addr:street:name" v="Lincoln"/>
  should be turned into
  {'id': 12345, 'key': 'street:name', 'value': 'Lincoln', 'type': 'addr'}

- If a node has no secondary tags then the "node_tags" field should just contain an empty list.

The fnameal return value for a "node" element should look something like:

{'node': {'id': 757860928,
          'user': 'uboot',
          'uid': 26299,
       'version': '2',
          'lat': 41.9747374,
          'lon': -87.6920102,
          'timestamp': '2010-07-22T16:16:51Z',
      'changeset': 5288876},
 'node_tags': [{'id': 757860928,
                'key': 'amenity',
                'value': 'fast_food',
                'type': 'regular'},
               {'id': 757860928,
                'key': 'cuisine',
                'value': 'sausage',
                'type': 'regular'},
               {'id': 757860928,
                'key': 'name',
                'value': "Shelly's Tasty Freeze",
                'type': 'regular'}]}

### If the element top level tag is "way":
The dictionary should have the format {"way": ..., "way_tags": ..., "way_nodes": ...}

The "way" field should hold a dictionary of the following top level way attributes:
- id
-  user
- uid
- version
- timestamp
- changeset

All other attributes can be ignored

The "way_tags" field should again hold a list of dictionaries, following the exact same rules as
for "node_tags".

Additionally, the dictionary should have a field "way_nodes". "way_nodes" should hold a list of
dictionaries, one for each nd child tag.  Each dictionary should have the fields:
- id: the top level element (way) id
- node_id: the ref attribute value of the nd tag
- position: the index starting at 0 of the nd tag i.e. what order the nd tag appears within
            the way element

The fnameal return value for a "way" element should look something like:

{'way': {'id': 209809850,
         'user': 'chicago-buildings',
         'uid': 674454,
         'version': '1',
         'timestamp': '2013-03-13T15:58:04Z',
         'changeset': 15353317},
 'way_nodes': [{'id': 209809850, 'node_id': 2199822281, 'position': 0},
               {'id': 209809850, 'node_id': 2199822390, 'position': 1},
               {'id': 209809850, 'node_id': 2199822392, 'position': 2},
               {'id': 209809850, 'node_id': 2199822369, 'position': 3},
               {'id': 209809850, 'node_id': 2199822370, 'position': 4},
               {'id': 209809850, 'node_id': 2199822284, 'position': 5},
               {'id': 209809850, 'node_id': 2199822281, 'position': 6}],
 'way_tags': [{'id': 209809850,
               'key': 'housenumber',
               'type': 'addr',
               'value': '1412'},
              {'id': 209809850,
               'key': 'street',
               'type': 'addr',
               'value': 'West Lexington St.'},
              {'id': 209809850,
               'key': 'street:name',
               'type': 'addr',
               'value': 'Lexington'},
              {'id': '209809850',
               'key': 'street:prefix',
               'type': 'addr',
               'value': 'West'},
              {'id': 209809850,
               'key': 'street:type',
               'type': 'addr',
               'value': 'Street'},
              {'id': 209809850,
               'key': 'building',
               'type': 'regular',
               'value': 'yes'},
              {'id': 209809850,
               'key': 'levels',
               'type': 'building',
               'value': '1'},
              {'id': 209809850,

               'key': 'building_id',
               'type': 'chicago',
               'value': '366409'}]}
"""

import csv
import codecs
import pprint
import re
import xml.etree.cElementTree as ET

import cerberus

import schema

OSM_PATH = "tampa_florida.osm"

NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

SCHEMA = schema.schema

# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']

def clean_tag(tag):
    if is_street_name(tag):
        audit_street_type(street_types, tag.attrib['v'])
    elif is_zipcode(tag):
        audit_zipcode(zipcode_types, tag.attrib['v'])
    elif is_state(tag):
        audit_state(tag.attrib['v'])
    elif is_city(tag):
        audit_city(city_types, tag.attrib['v'])
    else
        return 


def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  # Handle secondary tags the same way for both node and way elements    
    
    if element.tag == 'node':
        for attrib in element.attrib:
            if attrib in NODE_FIELDS:
                node_attribs[attrib] = element.attrib[attrib]
        for child in element:
            node_tag = {}
            if LOWER_COLON.match(child.attrib['k']):
                node_tag['type'] = child.attrib['k'].split(':',1)[0]
                node_tag['key'] = child.attrib['k'].split(':',1)[1]
                node_tag['id'] = element.attrib['id']
                node_tag['value'] = child.attrib['v']
                tags.append(node_tag)
            elif PROBLEMCHARS.match(child.attrib['k']):
                continue
            else:
                node_tag['type'] = 'regular'
                node_tag['key'] = child.attrib['k']
                node_tag['id'] = element.attrib['id']
                node_tag['value'] = child.attrib['v']
                tags.append(node_tag)
        
        return {'node': node_attribs, 'node_tags': tags}
        
    elif element.tag == 'way':
        for attrib in element.attrib:
            if attrib in WAY_FIELDS:
                way_attribs[attrib] = element.attrib[attrib]
        
        position = 0
        for child in element:
            way_tag = {}
            way_node = {}
            
            if child.tag == 'tag':
                if LOWER_COLON.match(child.attrib['k']):
                    way_tag['type'] = child.attrib['k'].split(':',1)[0]
                    way_tag['key'] = child.attrib['k'].split(':',1)[1]
                    way_tag['id'] = element.attrib['id']
                    way_tag['value'] = child.attrib['v']
                    tags.append(way_tag)
                elif PROBLEMCHARS.match(child.attrib['k']):
                    continue
                else:
                    way_tag['type'] = 'regular'
                    way_tag['key'] = child.attrib['k']
                    way_tag['id'] = element.attrib['id']
                    way_tag['value'] = child.attrib['v']
                    tags.append(way_tag)
                    
            elif child.tag == 'nd':
                way_node['id'] = element.attrib['id']
                way_node['node_id'] = child.attrib['ref']
                way_node['position'] = position
                position += 1
                way_nodes.append(way_node)
        
    return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}


# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_string = pprint.pformat(errors)
        
        raise Exception(message_string.format(field, error_string))


class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)


# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])


if __name__ == '__main__':
    # Note: Validation is ~ 10X slower. For the project consider using a small
    # sample of the map when validating.
    process_map(OSM_PATH, validate=True)


In [ ]:
process_map('tampa_florida.osm',False)

In [1]:
#%%writefile database.py
import csv
import sqlite3

conn = sqlite3.connect('tampa.db')
conn.text_factory = str
cur = conn.cursor()
'''
cur.execute('CREATE TABLE nodes (id INTEGER PRIMARY KEY NOT NULL, lat REAL, lon REAL, user TEXT, uid INTEGER, version INTEGER, changeset INTEGER, timestamp TEXT);')
with open('nodes.csv','rb') as fname:
    dr = csv.DictReader(fname)
    to_db = [(i['id'], i['lat'], i['lon'], i['user'], i['uid'], i['version'], i['changeset'], i['timestamp']) \
             for i in dr]

cur.executemany("INSERT INTO nodes (id, lat, lon, user, uid, version, changeset, timestamp) \
                VALUES (?, ?, ?, ?, ?, ?, ?, ?);", to_db)
conn.commit()

cur.execute("CREATE TABLE nodes_tags (id INTEGER, key TEXT, value TEXT, type TEXT, FOREIGN KEY (id) REFERENCES nodes(id));")
with open('nodes_tags.csv','rb') as fname:
    dr = csv.DictReader(fname) 
    to_db = [(i['id'], i['key'], i['value'], i['type']) for i in dr]

cur.executemany("INSERT INTO nodes_tags (id, key, value, type) VALUES (?, ?, ?, ?);", to_db)
conn.commit()

cur.execute("CREATE TABLE ways (id INTEGER PRIMARY KEY NOT NULL, user TEXT, uid INTEGER, version TEXT, changeset INTEGER, timestamp TEXT);")
with open('ways.csv','rb') as fname:
    dr = csv.DictReader(fname) 
    to_db = [(i['id'], i['user'], i['uid'], i['version'], i['changeset'], i['timestamp']) for i in dr]

cur.executemany("INSERT INTO ways (id, user, uid, version, changeset, timestamp) VALUES (?, ?, ?, ?, ?, ?);", to_db)
conn.commit()

cur.execute("CREATE TABLE ways_nodes (id INTEGER NOT NULL, node_id INTEGER NOT NULL, position INTEGER NOT NULL, FOREIGN KEY (id) REFERENCES ways(id), FOREIGN KEY (node_id) REFERENCES nodes(id));")
with open('ways_nodes.csv','rb') as fname:
    dr = csv.DictReader(fname) 
    to_db = [(i['id'], i['node_id'], i['position']) for i in dr]

cur.executemany("INSERT INTO ways_nodes (id, node_id, position) VALUES (?, ?, ?);", to_db)
conn.commit()

cur.execute("CREATE TABLE ways_tags (id INTEGER NOT NULL, key TEXT NOT NULL, value TEXT NOT NULL, type TEXT, FOREIGN KEY (id) REFERENCES ways(id));")
with open('ways_tags.csv','rb') as fname:
    dr = csv.DictReader(fname) 
    to_db = [(i['id'], i['key'], i['value'], i['type']) for i in dr]

cur.executemany("INSERT INTO ways_tags (id, key, value, type) VALUES (?, ?, ?, ?);", to_db)
conn.commit()
'''

'\ncur.execute(\'CREATE TABLE nodes (id INTEGER PRIMARY KEY NOT NULL, lat REAL, lon REAL, user TEXT, uid INTEGER, version INTEGER, changeset INTEGER, timestamp TEXT);\')\nwith open(\'nodes.csv\',\'rb\') as fname:\n    dr = csv.DictReader(fname)\n    to_db = [(i[\'id\'], i[\'lat\'], i[\'lon\'], i[\'user\'], i[\'uid\'], i[\'version\'], i[\'changeset\'], i[\'timestamp\'])              for i in dr]\n\ncur.executemany("INSERT INTO nodes (id, lat, lon, user, uid, version, changeset, timestamp)                 VALUES (?, ?, ?, ?, ?, ?, ?, ?);", to_db)\nconn.commit()\n\ncur.execute("CREATE TABLE nodes_tags (id INTEGER, key TEXT, value TEXT, type TEXT, FOREIGN KEY (id) REFERENCES nodes(id));")\nwith open(\'nodes_tags.csv\',\'rb\') as fname:\n    dr = csv.DictReader(fname) \n    to_db = [(i[\'id\'], i[\'key\'], i[\'value\'], i[\'type\']) for i in dr]\n\ncur.executemany("INSERT INTO nodes_tags (id, key, value, type) VALUES (?, ?, ?, ?);", to_db)\nconn.commit()\n\ncur.execute("CREATE TABLE ways (i

In [46]:
#%%writefile query.py

print "Number of nodes:"
print cur.execute('SELECT COUNT(*) FROM nodes').fetchone()[0]

print "Number of ways:"
print cur.execute('SELECT COUNT(*) FROM ways').fetchone()[0]

print "Number of unique users:"
print cur.execute('SELECT COUNT(DISTINCT(e.uid)) \
            FROM (SELECT uid FROM nodes UNION ALL SELECT uid FROM ways) e').fetchone()[0]

print "Top 10 contributing users:"
users = []
for row in cur.execute('SELECT e.user, COUNT(*) as num \
            FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) e \
            GROUP BY e.user \
            ORDER BY num DESC \
            LIMIT 10'):
    users.append(row)
print users

print "Number of users contributing once:"
cur.execute('SELECT COUNT(*) FROM \
                (SELECT e.user, COUNT(*) as num \
                 FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) e \
                 GROUP BY e.user \
                 HAVING num=1) u').fetchone()

print "Top 10 amenities:"
amenities = []
for row in cur.execute('SELECT value, COUNT(*) as num \
            FROM nodes_tags \
            WHERE key="amenity" \
            GROUP BY value \
            ORDER BY num DESC \
            LIMIT 10'):
    amenities.append(row)
print amenities

print "Top 5 places of worship:"
religions = []
for row in cur.execute('SELECT nodes_tags.value, COUNT(*) as num \
            FROM nodes_tags \
                JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value="place_of_worship") i \
                ON nodes_tags.id=i.id \
            WHERE nodes_tags.key="religion" \
            GROUP BY nodes_tags.value \
            ORDER BY num DESC \
            LIMIT 5;'):
    religions.append(row)
print religions

print "Top 5 cuisines"
cuisines = []
for row in cur.execute('SELECT nodes_tags.value, COUNT(*) as num \
            FROM nodes_tags \
                JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value="restaurant") i \
                ON nodes_tags.id=i.id \
            WHERE nodes_tags.key="cuisine" \
            GROUP BY nodes_tags.value \
            ORDER BY num DESC \
            LIMIT 5'):
    cuisines.append(row)
print cuisines

print "Top 10 restaurants:"
restaurants = []
for row in cur.execute('SELECT value, COUNT(*) as num \
            FROM nodes_tags \
                JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value="restaurant") i \
                ON nodes_tags.id=i.id \
            WHERE key="name"\
            GROUP BY value \
            ORDER BY num DESC \
            LIMIT 10'):
    restaurants.append(row)
print restaurants

Number of nodes:
1655566
Number of ways:
182866
Number of unique users:
1448
Top 10 contributing users:
[('coleman', 258302), ('woodpeck_fixbot', 235013), ('grouper', 187215), ('EdHillsman', 106677), ('NE2', 72924), ('David Hey', 60918), ('LnxNoob', 58364), ('KalininOV', 48825), ('westampa', 42145), ('bot-mode', 37656)]
Number of users contributing once:
Top 10 amenities:
[('restaurant', 852), ('place_of_worship', 771), ('school', 553), ('fast_food', 396), ('bicycle_parking', 353), ('bench', 279), ('fuel', 235), ('fountain', 201), ('bank', 170), ('toilets', 148)]
Top 5 places of worship:
[('christian', 724), ('jewish', 4), ('bahai', 3), ('buddhist', 3), ('unitarian_universalist', 3)]
Top 5 cuisines
[('american', 93), ('pizza', 70), ('mexican', 41), ('italian', 28), ('seafood', 25)]
Top 10 restaurants:
[('Tijuana Flats', 8), ("Applebee's", 6), ('Bob Evans', 6), ("Denny's", 6), ('IHOP', 6), ('Outback Steakhouse', 6), ('Panera Bread', 6), ("Chili's", 5), ('Golden Corral', 5), ('Pizza Hut'

In [49]:
cur.execute('SELECT value, count(*) \
            FROM nodes_tags WHERE value="restaurant"').fetchall()

[('restaurant', 852)]

In [48]:
cur.execute('SELECT value, count(*) \
            FROM nodes_tags JOIN (SELECT DISTINCT(id) \
            FROM nodes_tags WHERE value="restaurant") i \
                ON nodes_tags.id=i.id \
            WHERE key="delivery"\
            GROUP BY value').fetchall()

[('no', 72), ('yes', 19)]